In [17]:
import random
import json
import pickle
import numpy as np
import nltk


In [24]:
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import load_model

lemmatizer = WordNetLemmatizer
intents = json.loads(open('intents.json').read())

In [25]:
words= pickle.load(open('words.pkl', 'rb'))
classes= pickle.load(open('classes.pkl', 'rb'))
model = load_model('chatbot_model.keras')

In [26]:
def clean_up_sentence(sentence):
    sentence_words= nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

In [27]:
def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for w in sentence_words:
        for i,word in enumerate(words):
            if word == w:
                bag[i] ==1
    return np.array(bag)



In [29]:
def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i, r in enumerate(res)if r>ERROR_THRESHOLD]
    
    results.sort(key=lambda x: x[1],reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intents': classes[r[0]], 'proability': str[r[1]]})
    return return_list


In [31]:
def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intent = intents_json['intents']
    for i in list_of_intent:
        if i[tag] == tag:
            result = random.choice(i['response'])
            break
    return result



In [ ]:
print("Go! Bot is running!")

while True:
    message = input("You: ")
    if not message.strip():
        print("Bot: Please enter a valid message.")
        continue

    try:
        ints = predict_class(message)
        print("Debug: Predicted intents:", ints)

        res = get_response(ints, intents)
        print(f"Bot: {res}")
    except Exception as e:
        print(f"Bot: An error occurred: {str(e)}")
